In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import os
import sys
import re
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from Bio import SeqIO
from datetime import datetime
from numpy.random import randint
import torch
import torch.utils.data
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader

import matplotlib
import matplotlib.pyplot as plt

In [3]:
print(torch.__version__)
print(torch.cuda.device_count())
torch.cuda.is_available()

1.9.0+cu102
0


False

In [4]:
# Add Cuda availability

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader(DataLoader):
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device, batchSize, *args, **kwargs):
        super(DeviceDataLoader, self).__init__(dl, batchSize, *args, **kwargs)
        self.dl = dl
        self.device = device
        # for b in self.dl: 
        #   yield to_device(b, self.device)
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        # super(DeviceDataLoader, self).__iter__()
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
print(device)



cpu


In [5]:
k = 7

In [6]:
# !cp  '/content/drive/Shareddrives/FYP/Colab Notebooks/NEW_Approach/Python Files/WeightsCreator.py' '/content'
# !cp  '/content/drive/Shareddrives/FYP/Colab Notebooks/NEW_Approach/Python Files/HDF5dataset.py' '/content'
from HDF5dataset import HDF5Dataset
from WeightsCreator import make_weights_for_balanced_classes

In [7]:
inputFeatures = int((4**k) / 2)
# inputFeatures = 2
layer_array = [512, 512, 256, 256]
outputSize = 10
momentum = 0.4
dropoutProb = 0.5
batchSize = 400
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.001
num_workers = 4

In [8]:
# trainingDataset = HDF5Dataset('Plasmid_0.008_Dataset.h5', True)
#trainingDataset = HDF5Dataset('/home/chamikanandasiri/Test/Plasmid_0.008_Dataset.h5', True)
trainingDataset = HDF5Dataset('/home/chamikanandasiri/Datasets/DNAML_Plasmid/DNAML_plasmid_train.h5', True)

datasetsize = len(trainingDataset)
train_size = int(0.8 * len(trainingDataset))
val_size = len(trainingDataset) - train_size
                     
train_ds, val_ds = random_split(trainingDataset, [train_size, val_size])

0 data info added
5 data info added
10 data info added
15 data info added
20 data info added
25 data info added
30 data info added
35 data info added
40 data info added
45 data info added
1000 data info added
2000 data info added
3000 data info added
4000 data info added
5000 data info added
6000 data info added
7000 data info added
8000 data info added
9000 data info added
10000 data info added
11000 data info added
12000 data info added
13000 data info added
14000 data info added
15000 data info added
16000 data info added
17000 data info added
18000 data info added
19000 data info added
20000 data info added
21000 data info added
22000 data info added
23000 data info added
24000 data info added
25000 data info added
26000 data info added
27000 data info added
28000 data info added
29000 data info added
30000 data info added
31000 data info added
32000 data info added
33000 data info added
34000 data info added
35000 data info added
36000 data info added
37000 data info added
38000 d

In [9]:
#tens = torch.tensor([int(label) for _, label in train_ds])

t1 = datetime.now()
rtye = torch.tensor(train_ds.indices)
print(datetime.now()-t1)
tens = train_ds.dataset.get_label_values(rtye)
print(datetime.now()-t1)

0:00:00.012647
class counted in 0:00:00.000981
0 entries labels processed
5 entries labels processed
10 entries labels processed
15 entries labels processed
20 entries labels processed
25 entries labels processed
30 entries labels processed
35 entries labels processed
40 entries labels processed
45 entries labels processed
1000 entries labels processed
2000 entries labels processed
3000 entries labels processed
4000 entries labels processed
5000 entries labels processed
6000 entries labels processed
7000 entries labels processed
8000 entries labels processed
9000 entries labels processed
10000 entries labels processed
11000 entries labels processed
12000 entries labels processed
13000 entries labels processed
14000 entries labels processed
15000 entries labels processed
16000 entries labels processed
17000 entries labels processed
18000 entries labels processed
19000 entries labels processed
20000 entries labels processed
21000 entries labels processed
22000 entries labels processed
23

In [10]:
weights = make_weights_for_balanced_classes(outputSize, tens)
# weights = weights
# weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, datasetsize)

train_dl = DataLoader(train_ds, batchSize, shuffle=False, num_workers=num_workers, pin_memory=True, sampler=sampler)
val_dl = DataLoader(val_ds, batchSize, num_workers=num_workers, pin_memory=True)

# train_dl = DeviceDataLoader(train_ds, device, batchSize, shuffle=False, num_workers=num_workers, pin_memory=True, sampler=sampler)
# val_dl = DeviceDataLoader(val_ds, device, batchSize, num_workers=num_workers, pin_memory=True)

weight calculation started
class counted in 0:00:00.012564
class total in 0:00:00.023917
weight per class declared in 0:00:00.045548
weight per class calculated in 0:00:00.061468
weights declared in 0:00:00.063182
weights calculated in 0:00:03.141754


In [11]:
# index = 0
# for i in train_dl:
#   print("\n")
#   print(i[0])
#   print("\n")
#   print(i[1])
#   break

  
# train_dl = DeviceDataLoader(train_ds, device, batchSize, shuffle=False, num_workers=num_workers, pin_memory=True, sampler=sampler)
# val_dl = DeviceDataLoader(val_ds, device, batchSize, num_workers=num_workers, pin_memory=True)

# index = 0
# for i in train_dl:
#   print("\n")
#   print(i[0])
#   print("\n")
#   print(i[1])
#   break

# print(type(train_ds))

In [12]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

class Model(nn.Module):
    def __init__(self, in_size, layer_array = [512, 512, 256, 256], out_size = 28):
        super().__init__()
        self.network = nn.Sequential(
          nn.Linear(in_size, layer_array[0]),
          nn.ReLU(),
          # nn.Dropout(dropoutProb),
          nn.Linear(layer_array[0], layer_array[1]),
          nn.ReLU(),
          # nn.Dropout(dropoutProb),
          nn.Linear(layer_array[1], layer_array[2]),
          nn.ReLU(),
          # nn.Dropout(dropoutProb),
          nn.Linear(layer_array[2], layer_array[3]),
          nn.ReLU(),
          # nn.Dropout(dropoutProb),
          nn.Linear(layer_array[3], out_size)
        )
        
    def forward(self, xb):
        softmax = nn.LogSoftmax(dim=0)
        return softmax(self.network(xb))

    def training_step(self, batch):
        values, labels = batch 
        values = values.to(device)
        labels = labels.to(device)
        out = self(values)                  # Generate predictions
        loss = nn.functional.nll_loss(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        values, labels = batch 
        values = values.to(device)
        labels = labels.to(device)
        out = self(values)                    # Generate predictions
        loss = nn.functional.nll_loss(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

def predict_image(value, model):
  xb = value
  # //:TODO Check torch.stack
  # xb = torch.stack(value)
  yb = model(xb)
  _, preds  = torch.max(yb, dim=0)
  return preds

def predict_image(value, model):
    # Convert to a batch of 1
    xb = to_device(value, device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability   
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return dataset.classes[preds[0].item()]


In [13]:
model = Model(inputFeatures, layer_array, outputSize)
model = to_device(model, device)
model

Model(
  (network): Sequential(
    (0): Linear(in_features=8192, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=10, bias=True)
  )
)

In [ ]:
model.double()
t = datetime.now()
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)
print(datetime.now()-t)

0:01:40.396880\
0:02:31.303028

In [ ]:
plot_accuracies(history)

In [ ]:
plot_losses(history)

In [ ]:
predict_image(torch.from_numpy(train_ds([0][0]), model) , train_ds([0][1]))